# Solving Problems with Numerical Methods

The majority of courses in algebra, calculus, physics, chemistry, and engineering present us with problems that are difficult to formulate but straightforward to solve.  What I mean by that is we may struggle to develop the model equation that describes our system, but once we have that equation, it is usually easy for us to solve. Often, we are even able to solve that equation *analytically*. What do I mean by solving a problem analytically?  This is when I am able to use methods from algebra, trigonometry, calculus, etc. to "solve for x", i.e., to obtain a closed form solution that explicitly states the value of my variable of interest. Some examples to clarify that point:

$$0 = x^2 - 25$$

Can be solved by factoring or by simply moving 25 to the left hand side of the equation and taking the square root, giving:

$$x = 5 \ \textrm{or} \ x = -5$$
    
If not that, we could use a quadratic equation to get an explicit solution for x.

Another example from calculus:  Let's say I want to compute the area under the curve given by $y = 3x^3$ from x = 0 to x = 10.  It is straightforward to evaluate that definite integral:

$$Area = \int_0^{10}(3 x^3 dx) = \frac{3}{4} \cdot x^4 \big|_0^{10}$$

Which is evaluated as:

$$Area = \left(\frac{3}{4} \cdot 10^4\right) - \left(\frac{3}{4} \cdot 0^4\right) = 7500$$

Most of us are comfortable with these types of analytical solutions because it is what we've been trained to do. As we begin to deal with more advanced topics, we often need to solve problems where it may be tedious, difficult, or even impossible to find an analytical solution. This is where numerical methods come in.  Generally speaking, numerical methods are algorithms that employ simple operations (addition, subtraction, multiplication, and division) to provide approximate solutions to problems that do not have tractable analytical solutions. The basic operations at the core of numerical methods for root finding and differential equations solutions (again, generally simple addition, subtraction, multiplication, and division) are repeated iteratively, often many, many times, until we obtain a solution that is within our precision tolerance for a particular problem.

**The iterative nature of numerical methods makes them very tedious on paper but extremely efficient in most programming languages.**

When we first encounter numerical methods, many of us are uncomfortable with them because they look so different from the analytical solutions that have been ingrained in our minds as the way we will solve problems.  But fear not!  They are actually really simple.  We will see what is under the hood with numerical methods for a couple of test problems to get a feel for how they work, and then we'll introduce some packages available in Julia that handle numerical solutions for us in a very efficient and robust way.

Exercises on numerical methods will cover 3 broad classes of problems:

1. Root finding 
2. Optimization (finding minima and maxima)
3. Solving differential equations (initial value problems)
4. Numerical Quadrature (integrating a univariate function or discrete data set)

To build confidence, we'll do this in the way I always advocate when picking up a new problem solving technique: we will test it out on a problem that we already know how to solve to make sure we get the correct answer, and then we'll apply the methods to increasingly complex problems.

A final note about why you should embrace numerical methods:  Once you *get* them, they are really easy - often times they are *much* easier than developing an analytical solution.  You can write a code to handle them, and then reuse that code over and over and over by adapting it to suit the problem at hand.  They will greatly expand your ability to solve problems, and I guarantee that you will be able to use them in the rest of your engineering curricula.  

## Root Finding ##

Let's start with the most straightforward case first - using numerical methods to find the root of a function. Again, all we mean by root finding is finding the value of the function argument(s) where the function is equal to zero.  

### Univariate Functions ###

We will start with the case of root finding with a *univariate functions*, just means *a function that only has one independent variable*. For example, 

$$y(x) = x^2 + exp(x) - 7x + 5$$

is only a function of x.  Once I specify a value of x, I can evaluate the corresponding value of y.  y is thus a univariate function of x. We actually introduced a test univariate function in Assignment 04 (While Loops), and we used a simple numerical method to solve it, so let's pick up that example, use it again here, and expand our toolkit.

**The basic problem**

We have a *univariate* function, 

$$y = f(x) = 5x^2 + 8x - 23$$ 

We would like to find the **roots** for that function. By that, I mean the values of x where y = 0. So, when we discuss "root finding" for this equation, we are simply solving the equation below for x:

$$0 = 5x^2 + 8x - 23$$

This is an important thing to remember as we move forward. The equations that we see and develop in our models will take many different forms, but when we write programs that use numerical methods to solve them, for a univariate function, we will always be solving 0 = f(x), and so we will have to write our code accordingly.

**The Utility of Graphing your Functions**

Anytime I can do so (and I can typically do so with univariate functions), I will graph the function of interest on various domains (ranges of x values) so that I can see where roots might be located. As we consider the use of numerical methods, we will see how important it is to provide good initial guesses for the root or a reasonable range of x-values where we think we are likely to find the root. So if it is possible, start by graphing the function of interest.

We covered a bit of plotting during the review materials; let's start by defining a function that will calculate the value of the equation we are trying to solve at any value of x.  Specifically, we want to define the function of interest in the form of 0 = f(x). I will use a an inline function for this since it is a very simple example:

    f = x -> 5*x.^2 + 8*x .- 23

I can use that function to calculate the value of y for any value of x. You may notice that I've included the (.) operator in front of various operators.  This is so that I can pass a vector to the function and have it perform element-wise operations to return a vector of y values, i.e., my function is now *vectorized*.

Let's make a 1D array of x values that cover the domain of interest:

    xplot = collect(-10:0.25:10)

Now we can pass that column vector of x values into our function and have it return a column vector that contains the value of our equation at each x in the set.  We'll plot the values and see if we can figure out roughly where our function is equal to zero (i.e., where is the root). I'm adding a horizontal line at y = 0 to show where the function crosses zero. Just to get us used to formatting figures, we'll add some formatting options.

    using Plots
    yplot = f(xplot)
    plot(xplot, yplot, xlims = (-5, 5), ylims = (-50, 100), xticks = (-5:2.5:5), minorticks = 5)    #plot y vs x
    hline!([0], linecolor = :black)

I can see from that graph that there are two roots (which is the most we can have for a quadratic).  There is one between -4 and -3, and there is another between 1 and 2. We know from Supplement 06 (or just recalling our math background) that we can solve this problem exactly using the quadratic formula:

$$x = \frac{-8 \pm \sqrt{8^2 - 4\cdot5\cdot-23}}{2\cdot5}$$

Solving this, we find:

$$x = 1.489 \ \ \textrm{or} \ \ x = -3.089$$

This is absolutely consistent with our graphical analysis, as it should be! For me, this is a hugely important step when learning a new method:  we are establishing the correct answer using a method we are comfortable with *before* we start writing code to implement a method that is unfamiliar too us.

**A very crude numerical method**

You probably also recall that we wrote a while loop to solve this problem:

    x = 0.0                   #we need an initial guess for x
    y = -23.0                 #This is y at x = 0. We have to set an initial value for y
    while abs(y) > 0.1        #If abs(y) is greater than 0.1, we aren't close enough.  Keep going. 
        y = 5*x^2 + 8*x - 23  #calculate the value of y at current x
        println([x,y])        #Print result of iteration to see how loop is progressing
        x += 0.01             #increment x by 0.01 units.
    end
        
This is an inefficient code that automates a very uninformed guess-and-check iteration. At its core, you'll see there is no complex mathematical analysis: we guess a value for x, and we use simple operations to calculate the resulting value of y. We continue changing our value of x until we are close enough to y = 0 that we are satisfied with the solution. This is a very basic numerical method!

I don't advocate you use this one - it is crude in terms of tolerance, slow to converge, and rather unstable.  We really just used it to illustrate the function of a while loop. There are far better strategies for root finding. For example:

**Newton-Raphson Method**

A common entry point to numerical root finding is the Newton-Raphson Method. As with most numerical analysis, this is an iterative method. It uses information about the function value and the derivative value to make more informed iterations. The general "formula" for the Newton-Raphson iteration is:

$$x_{i+1} = x_i - \frac{f(x_i)}{f^\prime(x_i)}$$

In other words, you make a guess for x ($x_i$), you evaluate the function at that value of x, $f(x_i)$, and you evaluate the derivative of the function at that value of x, $f^\prime(x_i)$  You subract the function value divided by its derivative value from the current value of x, and that gives you the updated value of x. You proceed with this iteration until your function value is sufficiently close to zero that you are satisfied with the solution, i.e., you are within your precision tolerance. This algorithm is *much* more efficient than the simple guess and check method we used above.  We can write a Newton-Raphson iteration to solve our example problem. Because this is a far faster and more robust method, I will set a much tighter tolerance. You will notice that it takes far fewer iterations, and it converges to much closer to the value to the true root(s).  

    x = 0.0
    y = -23.0
    while abs(y) > 1e-8
        y  = 5*x^2 + 8*x - 23
        dy = 10*x + 8
        x  = x - y/dy
        println([x,y])
    end

Wow! I first learned the Newton-Raphson method in about 1999, where I had to write out several iterations by hand on green graph paper and solve it with a TI-89. After all these years, it still blows my mind how well it works.

A couple of things you should get used to: First, this method will return a single root.  The one that it finds is largely determined by my initial guess. If you know there are multiple roots, you can usually locate them by making different initial guesses in the range where you know the root to be - this is why starting with a graphical analysis is useful. Second, in the above example, I wrote out the expressions for y and the derivative of y inside the body of the while loop, but I could just as easily define them as external functions. This will be really handy in that we can test many methods on a single function (which we will do plenty of later in this exercise). I could easily do this using inline syntax, but I want to make sure we don't forget how to write long-form functions since they will be necessary for more complex problems.

    function yfun(x)
        5*x.^2 + 8*x .- 23
    end
     
    function dyfun(x)
        10*x + 8
    end

Now, I can write my while loop by calling those functions I just wrote:

    x = 0
    while abs(yfun(x)) > 1e-8
        x = x - yfun(x)/dyfun(x)
        println([x,yfun(x)])
    end
     
You should get used to that interchangability and how you can use an externally defined function over and over again throughout your code.

Finally, for a cool visualization of how iterative root finding algorithms proceed, run this code: It will give you a graphical output of the initial guess and the updated root location after each iteration. We start it out with a rather poor initial guess (x = 10), but eventually, you'll see that it settles in and stops at the value of the true root. As written, this plots the iterations in progess in an external windown, so be sure to switch over to it to watch how the iterations get closer to the true solution.

    xplot = collect(range(-4, stop = 10, length = 1000))
    yplot = yfun(xplot)
    xrange = [-4, 10]
    yrange = [0, 0]

    x = 10
    xarray = []
    yarray = []
    while abs(yfun(x)) > 1e-8
        xarray = [xarray;x]
        yarray = [yarray;yfun(x)]
        plt = plot(xplot, yplot)
        plot!(xrange,yrange,linestyle = :dot ,linewidth = 1)
        scatter!(xarray, yarray)
        display(plt)
        x = x - yfun(x)/dyfun(x);
        sleep(2)
    end

So that's a simple and efficient numerical method for root finding! It's elegant, and there isn't much complicated about it in practice!

We used basic operations (add, subtract, multiply, and divide) in an iterative loop to solve a nonlinear equation. Now, it is entertaining to write a Newton-Raphson method in a script. It's fine to do, but usually, I would advocate for taking advantage of methods available already in Julia.  First, they have been optimized by mathematicians and computer scientists, and so, in general, I would expect faster and more stable performance from an Julia root finding algorithm than one I have written from scratch. Second, the Newton-Raphson method is not always going to be the right choice. It is not guaranteed to converge, and it may give you poor stability in solving certain equations. The built-in root finding algorithms in Julia are going to be much more stable and robust

There are a few options for solving nonlinear equations in Julia. More often than not, we will be interested in finding the numerical value of the root--and we will frequently be solving equations that *cannot* be solved analytically.  So we will *usually* stick with numerial methods for root finding in this course.

When we have a univariate, scalar function that we are trying to solve, like y(x) = 0, the Roots package in Julia is what we want to use. It contains efficient algorithms for solving univariate scalar functions, namely *fzero* and *find_zero*. In Julia, when we need to add accessory packages, we'll have to install them:

    using Pkg
    Pkg.add("Roots")

Once we've installed Roots, we have to declare that we're using it:

    using Roots

Thereafter, we have access to solvers in the roots package.  The main ones you'll probably want are *fzero* and *find_zero*.  I'll use fzero for now since it should be familiar from Matlab.

    fzero(function_name, initial_guess)

You need the function itself, and you need an initial guess of what you think the correct root is.  That provides the starting point for the iterative method just as we did above.

For this particular example, all we provide is our function handle (y) and our initial guess at the root value (0):

    fzero(yfun, 0.0)

fzero is pretty simple; it returns the value of the root.  In general, it is a good habit to check and make sure when your function is evaluated at the root, it gives you zero:

    root1 = fzero(yfun, 0.0)
    fval  = yfun(root1)
    println(root1)
    print(fval)


You can set options in fzero or find_zero using various input sequences and keyword arguments.  

Type 

    ?fzero 
or 
    
    ?find_zero 

to get a feel for the options available.

It may sometimes be of interest to use non-default options on convergence tolerance or iteration progress.  You'll generally pass options as keywork arugments into the nlsolve() call.  There are far too many to cover fully here, but we'll go through some common examples.  

Let's set the tolerance on the change absolute tolerance in x values, atolX, to 1e-12. This controls how big of a change in the x value between iterations you are willing to accept at convergence. I will also change the maximum number of iterations (maxevals) to 500, and I'll change the function tolerance (how close we are to zero), atol, to 1e-12.  Finally, I'll enable verbose output.

    fzero(yfun, 0.0, xatol = 1e-12, atol = 1e-12, maxevals = 500, verbose = true)

We won't cover the full options available, but you have a lot of flexibility. In general, nlsolve root finding algorithms can probably handle whatever you have in mind.  You may occasionally need to tweak options or change the solver, but my gut feeling is that default behavior with fzero in Julia will be able to solve any nonlinear equations we hit during this course.  find_zero has a few more bells and whistles, including the ability to find all roots in a range, which may be useful in problems like CSTRs with heat effects, where there are typically many steady state solutions.

Now: some trouble.

The above example is really well-behaved. It is a robust 2nd order polynomial, and pretty much anything we use - graphical, quadratic, or any available numerical methods will solve it easily. Further, you can guess almost anything for your initial value of x, or provide reasonable brackets for the solver, and the algorithm will find the correct root.  This isn't always the case.  Some equations are more difficult to handle than others, and you may have to pay close attention to your initial guess or even your choice of solver.  This example is a bit contrived, but it will illustrate the point:

Consider the function:

$$g(t) = -0.74 + 0.765t + 1.1t^2 - 3.55t^3$$

When approaching a new problem that I will try to solve numerically, it is always helpful to graph it. Go ahead and do so to see if you can narrow down the range where you should look for a root.  

From the graph, I can see there is a root somewhere between -0.5 and -0.8.  Let's try to find it using our homebrew Newton-Raphson iteration, but we'll start with a bad initial guess on purpose. Run it and see what happens! With the Julia kernel, if you get stuck in a loop (you see [\*] to the left of the cell), you'll need to stop the code execution to break out of this. Now try again with different initial guesses and see if you can get it to converge on the root. 

    function gfun(t)
        -0.74 + 0.765*t + 1.1*t^2 - 3.55*t^3;
    end
    
    function dgfun(t)
        dgfun = 0.765 + 2.2*t - 3*3.55*t^2;
    end
    
    t =  5/9
    while abs(gfun(t)) > 1e-6
        t  = t - gfun(t)/dgfun(t)
        println([t,g(t)])
    end

Ok, so now let's see if Julia's fzero fares any better. Try finding the root of g(t) using fzero starting at the bad initial guess of 5/9. If it does not work, try a different initial guess to see if you can find the solution.

    fzero(gfun, 5/9)

You can see that Julia's algorithms natively do a pretty good job with this one. It uses an "Order0" method by default; I am not 100% sure what that algorithm is. Probably some sort of bracketing method, which tend to be pretty stable.  If you were to instead use a Newton method, I suspect it would not converge...

This is just to illustrate that solving nonlinear equations numerically can be quite difficult, and our best option is to use the algorithms written by professionals. We should also always remember that they can be very sensitive to our initial guess.  If one guess that you provide is not working, consider trying a different initial guess.  This is usually far more effective than trying to change how the solver works using options settings. For particularly difficult equations, you *may* need to change the solver, but I've rarely had this happen in reactor design courses.

### A great place for anonymous functions

Numerical root finding for univariate functions is a great place to get practice with writing and using anonymous (inline) functions.  For example, let's say I want to solve the function below using an initial guess of x = 10:

$$e^x = x^4 + 75.457$$

First, I need to make sure I convert this expression into a form 0 = f(x):

$$0 = x^4 - e^x + 75.457$$

Now, I *could* go through the usual, long form function definition of that function that I want to set equal to zero:
    
    function k(x)
        Fval = x^4 - exp(x) + 75.457
    end
    
    fzero(k,10)

But I don't necessarily need to formally assign that function handle since the function is so simple. This is a great place to use an anonymous function.  I have a few options.  I can use the inline format to assign the function handle:

    k1 = x -> (x.^4 - exp(x) + 75.457)
    fzero(k1, 10)  

Or I can dispense with assigning the function handle altogether and just use the anonymous function inline.

    fzero((x -> x^4 - exp(x) + 75.457), 10)
    
Take your pick! Whichever makes sense to you.  They all accomplish the same end result,and you may prefer one approach to another depending on specific problems you are solving.  My general rule of thumb is that I use long form function definitions for more complex functions that take multiple inputs and return multiple outputs, and I use inline function declarations for simple functions or when I need them for a special purpose (we'll see examples later).  

## Practice Problems ##

Find all of the roots/solutions for each of the examples below.

Remember: It may be useful to graph your functions so that you can provide good initial guesses to fzero.


$$f(x) = x^2 - 6x + 9$$

$$f(x) = x - \cos(x)$$

$$f(x) = e^xln(x^2) - x\cos(x)$$

$$e^x - 25x + 7 = x^3 - 2x - 2$$

$$\tanh(x) - 5 = \sinh(x) + 15x^2 + 23$$